In [50]:
import pandas as pd
import numpy as np
import json

In [3]:
# Importacion de Plotly como libreria de DataViz
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [4]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [41]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
df = pd.read_csv( path_base + r"\\data\procesed\gameplay_by_team.csv")

## Coeficientes de correlación de Pearson

Observa que los coeficientes de correlación de Spearman y Kendall son menos sensibles a relaciones no lineales y a valores atípicos, por lo que podrían ser más adecuados para conjuntos de datos en los que estas condiciones son una preocupación.

In [15]:
def plot_correlation_matrix_plotly(features, method='pearson'):
    correlation_matrix = features.corr(method=method)

    fig = px.imshow(correlation_matrix,
                    labels=dict(x="Features", y="Features", color=f"{method.capitalize()} Correlation"),
                    x=correlation_matrix.columns,
                    y=correlation_matrix.columns,
                    color_continuous_scale='balance',
                    width=800, height=800)

    fig.update_layout(title=f'Correlation Matrix ({method.capitalize()} Correlation)',
                      xaxis_title="Features",
                      yaxis_title="Features")

    fig.show()

In [45]:
def correlation_matrix(features, method='pearson'):
    correlation_matrix = features.corr(method=method)
    np.set_printoptions(precision=2, suppress=True)
    return correlation_matrix

In [43]:
def get_highly_correlated_features(correlation_matrix, threshold=0.6):
    # Filtrar las correlaciones que están por encima del umbral
    high_correlations = (correlation_matrix.abs() > threshold) & (correlation_matrix != 1)    
    # Obtener pares de características altamente correlacionadas
    pairs = np.column_stack(np.where(high_correlations))    
    # Crear listas de características altamente correlacionadas
    correlated_features = {}
    for pair in pairs:
        feature1, feature2 = correlation_matrix.columns[pair[0]], correlation_matrix.columns[pair[1]]
        if feature1 not in correlated_features:
            correlated_features[feature1] = [feature2]
        else:
            correlated_features[feature1].append(feature2)            
    return correlated_features

In [46]:
# Obtener la matriz de correlación
correlation_matrix = correlation_matrix(df, method='pearson')

# Obtener listas de características altamente correlacionadas
highly_correlated_features = get_highly_correlated_features(correlation_matrix, threshold=0.6)

# Imprimir las listas de características altamente correlacionadas
for feature, correlated_list in highly_correlated_features.items():
    print(f"{feature} está altamente correlacionada con: {', '.join(correlated_list)}")


Kills está altamente correlacionada con: Assists, TotalGold, GoldDiff, GoldPerMin
Deaths está altamente correlacionada con: GoldDiff
Assists está altamente correlacionada con: Kills, TotalGold, GoldPerMin
EliteMonsters está altamente correlacionada con: Dragons, Heralds
Dragons está altamente correlacionada con: EliteMonsters
Heralds está altamente correlacionada con: EliteMonsters
TotalGold está altamente correlacionada con: Kills, Assists, AvgLevel, TotalExperience, GoldDiff, ExperienceDiff, GoldPerMin
AvgLevel está altamente correlacionada con: TotalGold, TotalExperience, GoldDiff, ExperienceDiff, GoldPerMin
TotalExperience está altamente correlacionada con: TotalGold, AvgLevel, GoldDiff, ExperienceDiff, GoldPerMin
TotalMinionsKilled está altamente correlacionada con: CSPerMin
GoldDiff está altamente correlacionada con: Kills, Deaths, TotalGold, AvgLevel, TotalExperience, ExperienceDiff, GoldPerMin
ExperienceDiff está altamente correlacionada con: TotalGold, AvgLevel, TotalExperienc

C:\Users\Fernando\AppData\Local\Temp\ipykernel_12996\3116037376.py:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



## Consolidando Feature

### Conjunto de Características de Visión:
1. **Features**:
    - WardsPlaced
    - WardsDestroyed
    - EliteMonsters
    - Dragons
    - Heralds
    - TowersDestroyed
2. **Importancia**:
    - Proporciona información sobre el control del equipo en el mapa.
    - Puede influir en estrategias de rotación y control de objetivos.

In [6]:
# Conjunto de Características de Visión
vision_features = ['WardsPlaced', 'WardsDestroyed', 'EliteMonsters', 'Dragons', 'Heralds', 'TowersDestroyed']

In [16]:
plot_correlation_matrix_plotly(df[vision_features], method='pearson')

### Conjunto de Características de Rendimiento en Combate:
1. **Features**:
    - FirstBlood
    - Kills
    - Deaths
    - Assists
    - TotalGold
    - GoldDiff
    - ExperienceDiff
    - CSPerMin
    - GoldPerMin
2. **Importancia**:
    - Refleja el desempeño individual y colectivo en combate.
    - Puede influir en la acumulación de recursos y ventajas para el equipo.

In [7]:
# Conjunto de Características de Rendimiento en Combate
combat_features = ['FirstBlood', 'Kills', 'Deaths', 'Assists', 'TotalGold', 'GoldDiff', 'ExperienceDiff', 'CSPerMin', 'GoldPerMin']

In [17]:
plot_correlation_matrix_plotly(df[combat_features], method='pearson')

### Conjunto de Características de Desarrollo y Experiencia:
1. **Features**:
    - AvgLevel
    - TotalExperience
    - TotalMinionsKilled
    - TotalJungleMinionsKilled
2. **Importancia**:
    - Muestra el crecimiento y desarrollo de los campeones del equipo.
    - Puede influir en la capacidad del equipo para enfrentar desafíos más avanzados.

In [8]:
# Conjunto de Características de Desarrollo y Experiencia
development_features = ['AvgLevel', 'TotalExperience', 'TotalMinionsKilled', 'TotalJungleMinionsKilled']

In [23]:
plot_correlation_matrix_plotly(df[development_features], method='pearson')

### Conjunto de Características de Combate Temprano:
1. **Features**:
    - FirstBlood
    - Kills
    - Deaths
    - Assists
    - WardsPlaced
    - WardsDestroyed
2. **Importancia**:
    - Se centra en eventos tempranos que podrían influir en el curso del juego.
    - Puede revelar estrategias y decisiones cruciales en las etapas iniciales.

In [9]:
# Conjunto de Características de Combate Temprano
early_combat_features = ['FirstBlood', 'Kills', 'Deaths', 'Assists', 'WardsPlaced', 'WardsDestroyed']

In [24]:
plot_correlation_matrix_plotly(df[early_combat_features], method='kendall')

### Conjunto de Características Económicas:
1. **Features**:
    - TotalGold
    - GoldPerMin
    - GoldDiff
2. **Importancia**:
    - Evalúa la capacidad del equipo para acumular recursos económicos.
    - Puede reflejar ventajas económicas y desventajas durante el juego.

In [10]:
# Conjunto de Características Económicas
economic_features = ['TotalGold', 'GoldPerMin', 'GoldDiff']

In [20]:
plot_correlation_matrix_plotly(df[economic_features], method='pearson')

### Conjunto de Características de Desempeño en Dragones y Heralds:
1. **Features**:
    - Dragons
    - Heralds
    - GoldDiff
    - ExperienceDiff
2. **Importancia**:
    - Evalúa el éxito en la captura de objetivos importantes.
    - Puede influir en la ventaja estratégica y táctica del equipo.

In [11]:
# Conjunto de Características de Desempeño en Dragones y Heralds
objective_features = ['Dragons', 'Heralds', 'GoldDiff', 'ExperienceDiff']

In [21]:
plot_correlation_matrix_plotly(df[objective_features], method='pearson')

## Impacto en el Modelo:
- La elección de qué conjuntos de features utilizar dependerá de los objetivos específicos del modelo y del conocimiento del dominio del juego.
- Un modelo podría beneficiarse al considerar múltiples aspectos del juego, permitiendo capturar la complejidad de las interacciones entre las variables.
- La interpretación de los resultados podría ser más robusta al tener en cuenta diferentes dimensiones del rendimiento y estrategia del equipo.

## Feature Engineering 

1. Creación de Característica de KDA:
    - Podrías combinar las características de Kills, Deaths y Assists para crear una nueva característica que represente el rendimiento global de un jugador, como la relación de Kills y Assists sobre Deaths (KDA).

In [28]:
df['KDA'] = (df['Kills'] + df['Assists']) / df['Deaths'].replace(0, 1)

2. Normalización de Variables Económicas:
    - Normaliza las características relacionadas con el oro para asegurarte de que estén en la misma escala.

In [30]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['TotalGold', 'GoldDiff']] = scaler.fit_transform(df[['TotalGold', 'GoldDiff']])

3. Creación de Característica de Dominio - Torre Destroyed Ratio:
    - Puedes crear una característica que represente la eficiencia del equipo en destruir torres en relación con el total de torres disponibles.

In [31]:
df['TowerDestroyedRatio'] = df['TowersDestroyed'] / df['TowersDestroyed'].replace(0, 1)


In [33]:
plot_correlation_matrix_plotly(df[["KDA","TowerDestroyedRatio",'TotalGold', 'GoldDiff']], method='pearson')

#### Testeando correlaciones con caracteristicas creadas:

In [34]:
plot_correlation_matrix_plotly(df[["KDA"]+combat_features], method='pearson')

## Exportando Features

In [54]:
features = {
    "correlaciones": json.loads(correlation_matrix.to_json()),
    "grupos": [
        { "nombre": "Conjunto de Características de Visión", "variables": vision_features },
        { "nombre": "Conjunto de Características de Rendimiento en Combate", "variables": combat_features },
        { "nombre": "Conjunto de Características de Desarrollo y Experiencia", "variables": development_features },
        { "nombre": "Conjunto de Características de Combate Temprano", "variables": early_combat_features },
        { "nombre": "Conjunto de Características Económicas", "variables": economic_features },
        { "nombre": "Conjunto de Características de Desempeño en Dragones y Heralds", "variables": objective_features },
    ]}

In [56]:
with open(path_base + r"\\models\features.json", "w") as outfile:
    outfile.write(json.dumps(features, indent=4))